Link : https://www.kaggle.com/competitions/nlp-getting-started/overview/evaluation

## Blue Print

1. Check dataset
2. Cleaning
3. Preprocessing
4. Data Split (using stratified sampling)

## Error Function

$F_1 = 2\frac{precision * recall}{precision + recall}$ (1 is the best, 0 is the worst) where:
 
precision = $\frac{TP}{TP+FP}$, recall = $\frac{TP}{TP+FN}$

In [1]:
# sklearn.metrics.f1_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', 
# sample_weight=None, zero_division='warn')
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

## 1. Data Investigation

In [1]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score

In [3]:
# Load dataset
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print(train.shape, test.shape)
train.head()

(7613, 5) (3263, 4)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
# Types of values in each column
print(train.dtypes)

id           int64
keyword     object
location    object
text        object
target       int64
dtype: object


In [5]:
print(train[train["target"] == 0]["text"].values[10])       # not a disaster tweet
print(train[train["target"] == 1]["text"].values[1])  # disaster tweet

No way...I can't eat that shit
Forest fire near La Ronge Sask. Canada


## 2. Data Cleaning

In [5]:
# Check percentage of missing values
print("Ratio of missing values in training dataset")
print("missing keyword: ", str(round(train["keyword"].isnull().sum()/train.shape[0], 2)))
print("missing location: ", str(round(train["location"].isnull().sum()/train.shape[0], 2)), "\n")

print("Ratio of missing values in testing dataset")
print("missing keyword: ", str(round(test["keyword"].isnull().sum()/test.shape[0], 2)))
print("missing location: ", str(round(test["location"].isnull().sum()/test.shape[0], 2)))

Ratio of missing values in training dataset
missing keyword:  0.01
missing location:  0.33 

Ratio of missing values in testing dataset
missing keyword:  0.01
missing location:  0.34


=> Since the test dataset also contains missing values in 'keyword' and 'location', we will drop these columns and use 'text' column only.

## 3. Preprocessing

In [6]:
# Drop irrelevant columns
train = train.drop(["keyword", "location"], axis=1)
test = test.drop(["keyword", "location"], axis=1)

train.head()

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [28]:
# Data Split
X_train, X_test, y_train, y_test = train_test_split(train.loc[:, "id":"text"], train["target"], 
                                    test_size=0.3, random_state=0, stratify=train["target"])

print(X_train.shape, X_test.shape)

(5329, 2) (2284, 2)


In [31]:
# Store "id"
id_train = X_train["id"]
id_test = X_test["id"]
id_train.head()

4812    6849
2080    2988
652      944
3528    5043
3007    4321
Name: id, dtype: int64

In [37]:
# Convert a collection of text documents to a matrix of token counts
count_vectorizer = CountVectorizer(stop_words="english")
count_train = count_vectorizer.fit_transform(X_train["text"].values)
count_test = count_vectorizer.transform(X_test["text"].values)

print(count_train.shape)

(5329, 16619)


AttributeError: head not found

## 4. 

text preprocessing
- Word2Vec
- tweettoeknizer 
- tokenize -> lower -> Counter() and .most_common()
- remove stop words stopwords.word('english')? english_stops?